In [1]:
import pandas as pd 
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime 
from datetime import date
from tqdm import tqdm 
from dateutil.relativedelta import relativedelta
import scipy.fftpack
from scipy import signal
from scipy.signal import find_peaks
from scipy import signal

# Import DataSet from Hard Drive


In [2]:
def sensor_checkup(day, month, year, sensor_num = 'S6.1.1'): 
    sensor_num = sensor_num.upper()
    if 'S' in sensor_num:
        lut = pd.read_csv('values/S_tabel_'+str(year)+'_'+str(month) + '.csv')
        sensor_name = sensor_num.replace('.',"")
        if lut[sensor_name].values[day-1] == 1 & lut['Presence'].values[day-1] == 1: 
            return 1 
        else:  
            return 0
    else: 
        print("Inserted Sensor Name is Invlid")
        return None

In [3]:
# To upload one day 
def day_import(sensor_num = 'S6.1.3', gg = None, directory = None):
    ''' 
    start and finish date should be entered as 'date(2019,6,20)'. 
    '''
    ds = []
    pflag = 0
    y,m,d = gg.strftime('%Y,%m,%d').split(",")
    check = sensor_checkup(day = int(d), month = int(m), year = int(y) ,sensor_num = sensor_num)
    day = y + '-' + m + '-' + d
    if check == 1: 
        ds = pd.read_parquet(directory + 'ss335-acc'+day+sensor_num.replace('.','')+ '.parquet')
        pflag = 1
        return ds, pflag
    elif check == 0: 
        ds = None
        pflag = 0
        return ds, pflag
    


In [4]:
# upload a period of time 
def store_days(start = None, end = None, sensor_num = 's.611', directory = None): 
    '''
    Start and end day should be inserted as date(yy,mm,dd)
    '''
    
    output = {}
    flag = {}
    period = relativedelta(end, start).days
    period += 1
    print(str(period) + ' number of days are asked to be uploaded')
    for gg in tqdm(pd.date_range(start = start, end = end)): 
        y,m,d = gg.strftime('%Y,%m,%d').split(",")
        output[str(int(d))],flag[str(int(d))]  = day_import(sensor_num= sensor_num, gg = gg, 
                         directory=directory)
    return output, flag
    

In [5]:
#store summary of the uploaded data
def summary_store(pflag, start = None, end = None): 
    moment = datetime.now()
    mth = moment.month
    d = moment.day
    h = moment.hour
    mit = moment.minute
    period = relativedelta(end, start).days
    period += 1 
    y,m,d = start.strftime('%Y,%m,%d').split(",")
    file = open('summary of month_'+str(int(m))+' which stored at_'+str(d)+'_'+str(h)+'_'+str(mit)+'time.txt', 'w')

    for i in range(period): 
        comp = pflag[str(int(d)+ i)]
        if comp == 0: 
            file.write('day ' + str(int(d)+ i) + ' does NOT belong in the dataset for requiered sensor\n')
        elif comp == 1: 
            file.write('day ' + str(int(d)+ i) + ' is present in dataset\n')
    file.close()
    

In [6]:
# Concatenate discussed period
def conc_dict(ds, pflag, start = None, end = None, axis = 'z'): 
    output = {}
    start = start
    end = end
    y,m,d = start.strftime('%Y,%m,%d').split(",")

    size = relativedelta(end, start).days
    size += 1
    for i in range(size):
        comp = pflag[str(int(d)+i)]
        if comp == 0: 
            continue
        elif comp == 1:
            if output == {}:
                output[axis] = ds[str(int(d)+i)][axis].values
                output['ts'] = ds[str(int(d)+i)]['ts'].values
            else: 
                output[axis] = np.append(output[axis], ds[str(int(d)+i)][axis].values) 
                output['ts'] = np.append(output['ts'], ds[str(int(d)+i)]['ts'].values)   

    return output

In [7]:
# Gather All together 
def import_data(sens = 's611', axis = 'z', start = None, end = None, directory = None): 
    # First we store days in a dictionary (call from hard disk)
    pure_ds , flags = store_days(start = start, end = end, 
               sensor_num = sens, directory = directory)
    # Then we feed it to write a summary of what we upload: 
    #summary_store(flags, start = start, end = end)
    # Then we concatenate the axis we want in to one array: 
    print("Start to Merging Days of selected axis:")
    selected_ds = conc_dict(pure_ds, flags, start = start, end = end,
                            axis = axis)
    print("End of Merging\nSelected axis is: " + axis)
    # Return selected dataset for an axis + pure dataset
    return pure_ds, selected_ds

In [8]:
tic = datetime.now()
pure_may, sel_may = import_data(sens = 'S613', axis = 'z', 
                                  start = date(2019,5,10), end = date(2019,5,12),
                                  directory = 'values/')
tac = datetime.now()
print('Uploading time is: ' + str(tac - tic) + ' seconds')

100%|██████████| 3/3 [00:02<00:00,  1.24it/s]
Start to Merging Days of selected axis:
End of Merging
Selected axis is: z
Uploading time is: 0:00:03.404008 seconds


# Window Data For different Purposes 

In [9]:
def time_window(ds=None, axis = None, start = None, end= None): 
    ''' 
    Start and Finish should be inserted as dt.datetime(year,month,day,hour,minute,second).timestamp(
    axis also should be entered in this way => axis = 'z'
    '''
    output = {} 
    #define the upper boundary of the window
    output[axis] = ds[axis][(ds['ts']/1000)<end]
    output['ts'] = ds['ts'][(ds['ts']/1000)<end]
    #define the lower boundary of the window
    output[axis] = output[axis][(output['ts']/1000)>start]
    output['ts'] = output['ts'][(output['ts']/1000)>start]
    
    return output

In [10]:
RSNR = lambda x, xr: np.sum(x**2, axis=0)/np.sum((x - xr)**2, axis=0)
ARSNR = lambda x, xr: np.mean(RSNR(x, xr))
RSNRdB = lambda x, xr: 10*np.log10(RSNR(x, xr))
ARSNRdB = lambda x, xr: 10*np.log10(ARSNR(x, xr))

In [11]:
import datetime as dt
st1 = dt.datetime(2019,5,10,0,0,1).timestamp()
end1 = dt.datetime(2019,5,12,0,0,0).timestamp()
test = time_window(ds = sel_may, axis = 'z', start=st1, end=end1)



In [16]:
conv = (1*2.5)*2**-15
raw_data = test['z'][:200]
window1 = raw_data[:100]
window2 = raw_data[100:] 
"""
print(np.sum(window1**2, axis=0))
print(np.sum((window1 - window2)**2, axis=0))
print(ARSNRdB(window1, window2))
print(ARSNRdB(window1, window2))"""

'\nprint(np.sum(window1**2, axis=0))\nprint(np.sum((window1 - window2)**2, axis=0))\nprint(ARSNRdB(window1, window2))\nprint(ARSNRdB(window1, window2))'

In [22]:
def zscore(x): 
    print(x)
    print(np.mean(x))
    x = x-np.mean(x)
    print(x)
    return x/(np.std(x))
z_values_validate = zscore(window1)
print(z_values_validate)

[-13293 -13297 -13297 -13290 -13289 -13287 -13286 -13293 -13292 -13293
 -13286 -13284 -13291 -13299 -13298 -13291 -13295 -13291 -13280 -13287
 -13296 -13299 -13295 -13292 -13295 -13287 -13284 -13298 -13283 -13299
 -13272 -13301 -13304 -13301 -13294 -13295 -13297 -13290 -13293 -13302
 -13294 -13292 -13290 -13301 -13300 -13298 -13304 -13289 -13298 -13288
 -13296 -13292 -13294 -13292 -13296 -13289 -13287 -13294 -13289 -13288
 -13288 -13286 -13304 -13295 -13287 -13288 -13285 -13284 -13302 -13284
 -13303 -13289 -13300 -13289 -13292 -13297 -13276 -13290 -13300 -13286
 -13289 -13291 -13290 -13283 -13298 -13297 -13299 -13292 -13288 -13276
 -13279 -13283 -13299 -13293 -13289 -13292 -13287 -13288 -13285 -13289]
-13291.79
[ -1.21  -5.21  -5.21   1.79   2.79   4.79   5.79  -1.21  -0.21  -1.21
   5.79   7.79   0.79  -7.21  -6.21   0.79  -3.21   0.79  11.79   4.79
  -4.21  -7.21  -3.21  -0.21  -3.21   4.79   7.79  -6.21   8.79  -7.21
  19.79  -9.21 -12.21  -9.21  -2.21  -3.21  -5.21   1.79  -1.21 -1

In [40]:
import serial
import time

""" 
IMPORTANT: if your port is already open uncomment the following line. 
"""
#com.close()   

"""
Defining the com to send the data via UART
"""
com = serial.Serial("/dev/ttyUSB0", baudrate= 115200, 
                    parity = serial.PARITY_NONE, 
                    stopbits = serial.STOPBITS_ONE, 
                    bytesize = serial.EIGHTBITS,
                    timeout = 100000)
#com.open()

com.isOpen()
i = 0
"""
For consistency of comparisons in all 3 dimensions: 8
    - interval of 1 sec is 9600 - 9700 .
    - interval of 2 sec is 9600 - 9800
    - interval of 5 sec is 9500 - 10000
"""
for aray in raw_data[:100]:
    i  = i+1
    # To have 8 bits of data for each value, following reshaping is vital 
    if aray >0: 
        string = '+' + str(aray) +'\r\n'
         
    else:
        string = str(aray) + '\r\n'
          
    com.write((string).encode('ascii'))
    if i % 100 == 0:
        time.sleep(10)
    else:
        time.sleep(0.05) # to avoid data loss
com.close()

In [ ]:
com.close()

In [19]:
conv = (1*2.5)*2**-15
values = test['z']
scaled_values = values * conv    # To Verify the data

In [13]:
import serial
import time

""" 
IMPORTANT: if your port is already open uncomment the following line. 
"""
#com.close()   

"""
Defining the com to send the data via UART
"""
com = serial.Serial("com5", baudrate= 115200, 
                    parity = serial.PARITY_NONE, 
                    stopbits = serial.STOPBITS_ONE, 
                    bytesize = serial.EIGHTBITS,
                    timeout = 100000)
#com.open()
com.isOpen()
i = 0
"""
For consistency of comparisons in all 3 dimensions: 
    - interval of 1 sec is 9600 - 9700 
    - interval of 2 sec is 9600 - 9800
    - interval of 5 sec is 9500 - 10000
"""
for aray in values[9500:10000]:
    i  = i+1
    # To have 8 bits of data for each value, following reshaping is vital 
    if aray >0: 
        string = '+' + str(aray) +'\r\n' 
    else:
        string = str(aray) + '\r\n' 
    com.write((string).encode('ascii'))
    time.sleep(0.05) # to avoid data loss
com.close()

# MSE Verification

In [14]:
pca_reconst = lambda data,eigenvector: np.linalg.multi_dot([data,eigenvector, eigenvector.T])

In [15]:
# Helpful FoM to evaluate reconstruction of PCA 
RSNR = lambda x, xr: np.sum(x**2, axis=0)/np.sum((x - xr)**2, axis=0)
ARSNR = lambda x, xr: np.mean(RSNR(x, xr))
RSNRdB = lambda x, xr: 10*np.log10(RSNR(x, xr))
ARSNRdB = lambda x, xr: 10*np.log10(ARSNR(x, xr))
### ==================================================
mse_w = lambda x, xr: np.mean((x - xr)**2, axis=0)
mse = lambda x, xr: np.mean(mse_w(x, xr))
energy = lambda x: np.sum(x**2, axis=0)/500

In [16]:
eigen_val = pd.read_csv('eigen_values/eigenVal_1sec.csv')  # here put the name of the desired file that you want
eigen_val = eigen_val.to_numpy()

In [24]:
def time_shape(ds, input_dim):
    d = input_dim
    n = ds.shape[0]
    n0 = n // input_dim
    data = ds[:n0*input_dim]
    data = data.reshape((n0,d)).T
    mean_numero = np.mean(data, axis = 0)
    data_mean = data - np.mean(data, axis = 0)
    energy = np.sum(data_mean**2, axis = 0)/input_dim
    return data, data_mean, energy

In [25]:
# input dimension should be inserted as integer e.g 1,2,5,etc.
ds_py, ds_mean, ds_eg = time_shape(scaled_values, 100) 

In [26]:
a = np.where(ds_eg > 3.125e-06) 
print(a[0][:25])

[  96   97   98  445  446  902  903  904  905 1039 1040 1041 1042 1043
 1044 1045 1046 1085 1086 1087 1090 1091 1117 1118 1119]


In [27]:
import sklearn.preprocessing
"""
Corresponding number for former defined intervals are: 
    - 1 sec is 96
    - 2 sec is 48
    - 5 sec is 19
"""
scaled_data = sklearn.preprocessing.scale(ds_mean[:,96], with_mean=True, with_std=True, copy=True)
reco = pca_reconst(scaled_data,eigen_val)

In [28]:
sub = (reco - scaled_data)**2

In [29]:
print(np.mean(sub))

0.6358322155436077


# END OF CODE 